# Optimize Initial Conditions
## Brute Force

In [1]:
from scipy.optimize import brute,optimize
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
import ray

In [2]:
LoadBH=True
if LoadBH:
    import LearnerICRayNoLoadBH as L
else:
    import LearnerICRayNoLoad as L

In [3]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

In [4]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [5]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-06,SE,TOTAL,state,8972,208,74,True,2298696.0,28.0,390.30824,0.0232
1,1,2020-06-05,SE,TOTAL,state,8577,198,73,False,2298696.0,28.0,373.12459,0.0231
2,2,2020-06-04,SE,TOTAL,state,8200,186,72,False,2298696.0,28.0,356.72399,0.0227
3,3,2020-06-03,SE,TOTAL,state,7989,180,71,False,2298696.0,28.0,347.54487,0.0225
4,4,2020-06-02,SE,TOTAL,state,7555,172,70,False,2298696.0,28.0,328.66460,0.0228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,158210,2020-03-11,BA,TOTAL,state,3,0,5,False,14873064.0,29.0,0.02017,0.0000
2245,158211,2020-03-10,BA,TOTAL,state,2,0,4,False,14873064.0,29.0,0.01345,0.0000
2246,158212,2020-03-09,BA,TOTAL,state,2,0,3,False,14873064.0,29.0,0.01345,0.0000
2247,158213,2020-03-07,BA,TOTAL,state,2,0,2,False,14873064.0,29.0,0.01345,0.0000


In [6]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=2)
def opt(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
    optimal = brute(fobjective,        
        ranges=[slice(0.8*s0,1.2*s0,0.4/3*s0),slice(-4,4,2),slice(0.8*i0,1.2*i0,0.1*i0),\
                slice(0.3,0.6,0.1),slice(0.1,0.15,0.05)],
        args=(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version), \
        full_output=True, disp=True, finish=optimize.fmin)
    return optimal[0]

In [7]:
from scipy.integrate import odeint
import os
import sys
import io
import gc

def append_new_line(file_name, text_to_append):
    #Append given text as a new line at the end of file
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(9999999)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

def fobjective(point,state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
    
    dead =  load_dead(state,date)
    data = load_confirmed(state,date)*(1-ratio)-dead

    cleanRecovered=False
    s0, deltaDate, i0, weigthCases, weigthRecov = point
    end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=+int(deltaDate))

    f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                       s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                       cleanRecovered, version, data, dead, savedata=False)
    result = f.train.remote() 
    result = ray.get(result) 

    del end_date,cleanRecovered, data, dead, point         

    gc.collect()

    return result

In [8]:
states=['SP','ES','MG','RJ','PE','CE','AM']
allStates=True
version="10"
ray.shutdown()
GB=1024*1024*1024
MB=1024*1024
ray.init(object_store_memory=500*MB,memory=1*GB,lru_evict=True,\
         driver_object_store_memory=50*MB,num_cpus=10)
gc.enable()cd ..

runID=[]
if allStates:
    for state in states:
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        runID.append(opt.remote(state,s0,i0,e0,a0,r0,d0,startdate,startNCases, \
                                                    ratio, predict_range, version))
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    runID.append(opt.remote(state,s0,i0,e0,a0,r0,d0,startdate,startNCases, \
                                                    ratio, predict_range, version))

2020-06-14 22:57:14,553	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 0.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-14 22:57:14,895	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [ ]:
optimal=ray.get(runID)

(pid=23958) basinhopping step 0: f 1.12102e+08
(pid=23955) basinhopping step 0: f 6.2945e+07
(pid=23962) basinhopping step 0: f -1.62778e+13
(pid=23958) basinhopping step 1: f 1.11484e+08 trial_f 1.11484e+08 accepted 1  lowest_f 1.11484e+08
(pid=23958) found new global minimum on step 1 with function value 1.11484e+08
(pid=24018) basinhopping step 0: f -3.02658e+11
(pid=23962) basinhopping step 1: f -1.62778e+13 trial_f 3.07464e+11 accepted 0  lowest_f -1.62778e+13
(pid=24037) basinhopping step 0: f -1.47702e+11
(pid=23958) basinhopping step 2: f 1.11484e+08 trial_f 2.27323e+09 accepted 0  lowest_f 1.11484e+08
(pid=23958) basinhopping step 3: f 1.11484e+08 trial_f 1.81991e+11 accepted 0  lowest_f 1.11484e+08
(pid=23955) basinhopping step 1: f 6.25978e+07 trial_f 6.25978e+07 accepted 1  lowest_f 6.25978e+07
(pid=23955) found new global minimum on step 1 with function value 6.25978e+07
(pid=24121) basinhopping step 0: f -2.68919e+11
(pid=23958) basinhopping step 4: f 1.11484e+08 trial_f 

(pid=24121) basinhopping step 8: f -5.45537e+11 trial_f 1.1007e+08 accepted 0  lowest_f -5.45537e+11
(pid=23962) basinhopping step 10: f -1.62778e+13 trial_f 5.47263e+08 accepted 0  lowest_f -1.62778e+13
(pid=23958) basinhopping step 9: f -2.45004e+11 trial_f -1.45219e+11 accepted 0  lowest_f -2.45004e+11
(pid=24121) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=24121)   warnings.warn(warning_msg, ODEintWarning)
(pid=24129) basinhopping step 9: f -1.4378e+11 trial_f 5.48762e+07 accepted 0  lowest_f -1.4378e+11
(pid=24141) basinhopping step 1: f 7.09191e+07 trial_f 7.09191e+07 accepted 1  lowest_f 7.09191e+07
(pid=24141) found new global minimum on step 1 with function value 7.09191e+07
(pid=24121) warning: basinhopping: local minimization failure
(pid=24121) basinhopping step 9: f -5.45537e+11 trial_f -3.92688e+11 acc

(pid=24122) basinhopping step 6: f -3.57809e+11 trial_f 6.82644e+10 accepted 0  lowest_f -3.57809e+11
(pid=24122) basinhopping step 7: f -3.57809e+11 trial_f 3.63311e+11 accepted 0  lowest_f -3.57809e+11
(pid=24126) basinhopping step 9: f -3.12353e+11 trial_f -3.12353e+11 accepted 1  lowest_f -3.12353e+11
(pid=24126) found new global minimum on step 9 with function value -3.12353e+11
(pid=24128) basinhopping step 5: f -1.47185e+11 trial_f 4.1196e+07 accepted 0  lowest_f -1.47185e+11
(pid=24127) basinhopping step 4: f -1.25138e+11 trial_f 1.85071e+08 accepted 0  lowest_f -1.25138e+11
(pid=24128) basinhopping step 6: f -1.47185e+11 trial_f 9.30655e+08 accepted 0  lowest_f -1.47185e+11
(pid=24122) basinhopping step 8: f -3.57809e+11 trial_f 1.72367e+08 accepted 0  lowest_f -3.57809e+11
(pid=24127) warning: basinhopping: local minimization failure
(pid=24127) basinhopping step 5: f -1.25138e+11 trial_f 1.2544e+11 accepted 0  lowest_f -1.25138e+11
(pid=24128) warning: basinhopping: local mi

(pid=24414) basinhopping step 5: f -2.62393e+11 trial_f 1.79375e+08 accepted 0  lowest_f -2.62393e+11
(pid=24125) basinhopping step 5: f -9.17708e+11 trial_f 8.32857e+08 accepted 0  lowest_f -9.17708e+11
(pid=24486) basinhopping step 5: f -1.22029e+11 trial_f -1.22029e+11 accepted 1  lowest_f -1.22029e+11
(pid=24486) found new global minimum on step 5 with function value -1.22029e+11
(pid=24134) basinhopping step 8: f -1.24484e+11 trial_f -1.04824e+09 accepted 0  lowest_f -1.24484e+11
(pid=24125) basinhopping step 6: f -9.17708e+11 trial_f 1.5445e+13 accepted 0  lowest_f -9.17708e+11
(pid=24414) basinhopping step 6: f -2.62393e+11 trial_f 1.78612e+08 accepted 0  lowest_f -2.62393e+11
(pid=24134) basinhopping step 9: f -1.24484e+11 trial_f 1.85753e+08 accepted 0  lowest_f -1.24484e+11
(pid=24486) basinhopping step 6: f -1.22029e+11 trial_f 2.0268e+09 accepted 0  lowest_f -1.22029e+11
(pid=24415) basinhopping step 8: f -7.77913e+11 trial_f -3.07056e+11 accepted 0  lowest_f -7.77913e+11
(

(pid=24733) basinhopping step 5: f 8.37193e+07 trial_f 1.70876e+08 accepted 0  lowest_f 8.37193e+07
(pid=24485) basinhopping step 4: f -4.66858e+11 trial_f -4.55193e+10 accepted 0  lowest_f -4.66858e+11
(pid=24651) basinhopping step 6: f -3.56701e+11 trial_f 4.0535e+09 accepted 0  lowest_f -3.56701e+11
(pid=24652) warning: basinhopping: local minimization failure
(pid=24652) basinhopping step 6: f -3.10393e+11 trial_f 8.01282e+07 accepted 0  lowest_f -3.10393e+11
(pid=24651) basinhopping step 7: f -7.81586e+11 trial_f -7.81586e+11 accepted 1  lowest_f -7.81586e+11
(pid=24651) found new global minimum on step 7 with function value -7.81586e+11
(pid=24651) warning: basinhopping: local minimization failure
(pid=24651) basinhopping step 8: f -7.81586e+11 trial_f 1.29681e+11 accepted 0  lowest_f -7.81586e+11
(pid=24733) basinhopping step 6: f 8.37193e+07 trial_f 1.07507e+09 accepted 0  lowest_f 8.37193e+07
(pid=24721) basinhopping step 7: f 2.01722e+07 trial_f 1.76132e+08 accepted 0  lowest

...

(pid=111452) basinhopping step 0: f -2.89244e+11
(pid=111452) basinhopping step 1: f -2.89244e+11 trial_f -1.60657e+11 accepted 0  lowest_f -2.89244e+11
(pid=110258) basinhopping step 9: f -5.66296e+11 trial_f 3.03742e+06 accepted 0  lowest_f -5.66296e+11
(pid=108918) basinhopping step 8: f -7.4975e+13 trial_f -7.4975e+13 accepted 1  lowest_f -7.4975e+13
(pid=108918) found new global minimum on step 8 with function value -7.4975e+13
(pid=110258) basinhopping step 10: f -5.66296e+11 trial_f 7.914e+07 accepted 0  lowest_f -5.66296e+11
(pid=110257) warning: basinhopping: local minimization failure
(pid=110257) basinhopping step 7: f -2.6267e+11 trial_f -2.6267e+11 accepted 1  lowest_f -2.6267e+11
(pid=110257) found new global minimum on step 7 with function value -2.6267e+11
(pid=108918) basinhopping step 9: f -7.4975e+13 trial_f 2.91585e+09 accepted 0  lowest_f -7.4975e+13
(pid=110257) basinhopping step 8: f -2.6267e+11 trial_f 1.97481e+10 accepted 0  lowest_f -2.6267e+11
(pid=111831) ba

(pid=111831) basinhopping step 5: f -5.70525e+11 trial_f 8.2961e+07 accepted 0  lowest_f -5.70525e+11
(pid=112318) basinhopping step 6: f -5.44259e+11 trial_f 2.83022e+08 accepted 0  lowest_f -5.44259e+11
(pid=112318) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=112318)   warnings.warn(warning_msg, ODEintWarning)
(pid=112319) basinhopping step 4: f -7.59169e+13 trial_f 5.33123e+11 accepted 0  lowest_f -7.59169e+13
(pid=111451) basinhopping step 8: f -1.40968e+12 trial_f 2.9624e+08 accepted 0  lowest_f -1.40968e+12
(pid=111832) basinhopping step 7: f -5.7008e+11 trial_f -5.57123e+11 accepted 0  lowest_f -5.7008e+11
(pid=111451) basinhopping step 9: f -1.40968e+12 trial_f 1.52355e+08 accepted 0  lowest_f -1.40968e+12
(pid=111452) basinhopping step 10: f -7.42643e+11 trial_f 2.29109e+09 accepted 0  lowest_f -7.42643e+11

(pid=112682) basinhopping step 8: f -2.92279e+11 trial_f 1.50037e+08 accepted 0  lowest_f -2.92279e+11
(pid=111831) basinhopping step 10: f -6.33982e+11 trial_f -4.73884e+11 accepted 0  lowest_f -6.33982e+11
(pid=114436) basinhopping step 6: f -6.65037e+11 trial_f 9.55721e+07 accepted 0  lowest_f -6.65037e+11
(pid=112682) basinhopping step 9: f -2.92279e+11 trial_f 4.97338e+09 accepted 0  lowest_f -2.92279e+11
(pid=114435) basinhopping step 2: f -1.03743e+11 trial_f 7.71426e+07 accepted 0  lowest_f -1.03743e+11
(pid=114435) basinhopping step 3: f -1.03743e+11 trial_f 1.64166e+09 accepted 0  lowest_f -1.03743e+11
(pid=113842) basinhopping step 8: f -5.54987e+11 trial_f 6.23723e+07 accepted 0  lowest_f -5.54987e+11
(pid=112682) basinhopping step 10: f -9.17846e+11 trial_f -9.17846e+11 accepted 1  lowest_f -9.17846e+11
(pid=112682) found new global minimum on step 10 with function value -9.17846e+11
(pid=114436) basinhopping step 7: f -6.65037e+11 trial_f 1.09972e+08 accepted 0  lowest_f 

(pid=115472) basinhopping step 10: f -2.20604e+11 trial_f 8.37195e+07 accepted 0  lowest_f -2.20604e+11
(pid=114994) basinhopping step 9: f -2.21422e+11 trial_f 1.01876e+08 accepted 0  lowest_f -2.21422e+11
(pid=116604) basinhopping step 0: f 1.04497e+08
(pid=114993) basinhopping step 6: f -3.36392e+13 trial_f -2.94076e+13 accepted 0  lowest_f -3.36392e+13
(pid=115875) basinhopping step 2: f -5.42049e+11 trial_f 1.46706e+06 accepted 0  lowest_f -5.42049e+11
(pid=115875) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=115875)   warnings.warn(warning_msg, ODEintWarning)
(pid=115876) basinhopping step 7: f -6.66193e+10 trial_f 1.45642e+08 accepted 0  lowest_f -6.66193e+10
(pid=115875) basinhopping step 3: f -5.42049e+11 trial_f 1.04078e+11 accepted 0  lowest_f -5.42049e+11
(pid=114994) basinhopping step 10: f -2.21422e+11 

(pid=118135) basinhopping step 1: f -2.89365e+13 trial_f -2.88356e+13 accepted 0  lowest_f -2.89365e+13
(pid=118135) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=118135)   warnings.warn(warning_msg, ODEintWarning)
(pid=116604) basinhopping step 8: f -5.00266e+11 trial_f 1.84925e+09 accepted 0  lowest_f -5.00266e+11
(pid=117140) basinhopping step 3: f -2.61275e+11 trial_f 5.62547e+07 accepted 0  lowest_f -2.61275e+11
(pid=116604) basinhopping step 9: f -5.00266e+11 trial_f 1.04072e+08 accepted 0  lowest_f -5.00266e+11
(pid=116746) basinhopping step 9: f -5.69803e+11 trial_f 1.19742e+08 accepted 0  lowest_f -5.69803e+11
(pid=117142) basinhopping step 5: f -2.60151e+11 trial_f -2.60151e+11 accepted 1  lowest_f -2.60151e+11
(pid=117142) found new global minimum on step 5 with function value -2.60151e+11
(pid=118135) basi

In [ ]:

for i in range(0,len(states)):
    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")
    print(state[i]+" State of Brazil")
    print("S0 = {}\n".format(optimal[i][0]))
    print("Delta Date Days = {}\n".format(optimal[i][1]))   
    print("I0 = {}\n".format(optimal[i][2]))
    print("wcases = {}\n".format(optimal[i][3]))
    print("wrec = {}\n".format(optimal[i][4]))
    
    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]
    dfparam.to_csv("data/param_optimized_Brute_Force.csv", sep=",", index=False)
    dfparam